In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np
import json

# Wczytanie danych z pliku JSON
file_path = 'data/data.json'  # Upewnij się, że ścieżka jest poprawna
with open(file_path, 'r') as f:
    data = json.load(f)

# Parsowanie danych (dostosuj do formatu pliku)
df = pd.DataFrame(data)

# Sprawdzenie struktury danych
df.head()


,17296344,17298072,17298936,17299800,17300700,17304156,17305884,17306748,17307612,17308476,...,17386236,17387100,17387964,17388828,17389692,17390556,17391420,17393148,17394012,17413884
0,"{'id': 1, 'name': 'Breakfast', 'meals': [{'mea...","{'id': 1, 'name': 'Breakfast', 'meals': [{'mea...","{'id': 1, 'name': 'Breakfast', 'meals': [{'mea...","{'id': 1, 'name': 'Breakfast', 'meals': [], 'c...","{'id': 1, 'name': 'Breakfast', 'meals': [{'mea...","{'id': 1, 'name': 'Breakfast', 'meals': [{'mea...","{'id': 1, 'name': 'Breakfast', 'meals': [], 'c...","{'id': 1, 'name': 'Breakfast', 'meals': [{'mea...","{'id': 1, 'name': 'Breakfast', 'meals': [{'bas...","{'id': 1, 'name': 'Breakfast', 'meals': [{'bas...",...,"{'id': 1, 'name': 'Breakfast', 'meals': [{'bas...","{'id': 1, 'name': 'Breakfast', 'meals': [{'bas...","{'id': 1, 'name': 'Breakfast', 'meals': [{'mea...","{'id': 1, 'name': 'Breakfast', 'meals': [{'bas...","{'id': 1, 'name': 'Breakfast', 'meals': [{'mea...","{'id': 1, 'name': 'Breakfast', 'meals': [{'bas...","{'id': 1, 'name': 'Breakfast', 'meals': [{'bas...","{'id': 1, 'name': 'Breakfast', 'meals': [{'bas...","{'id': 1, 'name': 'Breakfast', 'meals': [{'bas...","{'id': 1, 'name': 'Breakfast', 'meals': [], 'c..."
1,"{'id': 2, 'name': 'Snack I', 'meals': [], 'cal...","{'id': 2, 'name': 'Snack I', 'meals': [], 'cal...","{'id': 2, 'name': 'Snack I', 'meals': [], 'cal...","{'id': 2, 'name': 'Snack I', 'meals': [{'mealN...","{'id': 2, 'name': 'Snack I', 'meals': [], 'cal...","{'id': 2, 'name': 'Snack I', 'meals': [], 'cal...","{'id': 2, 'name': 'Snack I', 'meals': [], 'cal...","{'id': 2, 'name': 'Snack I', 'meals': [{'base'...","{'id': 2, 'name': 'Snack I', 'meals': [{'grams...","{'id': 2, 'name': 'Snack I', 'meals': [{'grams...",...,"{'id': 2, 'name': 'Snack I', 'meals': [], 'cal...","{'id': 2, 'name': 'Snack I', 'meals': [{'base'...","{'id': 2, 'name': 'Snack I', 'meals': [{'base'...","{'id': 2, 'name': 'Snack I', 'meals': [{'base'...","{'id': 2, 'name': 'Snack I', 'meals': [], 'cal...","{'id': 2, 'name': 'Snack I', 'meals': [{'grams...","{'id': 2, 'name': 'Snack I', 'meals': [{'mealN...","{'id': 2, 'name': 'Snack I', 'meals': [{'grams...","{'id': 2, 'name': 'Snack I', 'meals': [], 'cal...","{'id': 2, 'name': 'Snack I', 'meals': [], 'cal..."
2,"{'id': 3, 'name': 'Lunch', 'meals': [], 'calcu...","{'id': 3, 'name': 'Lunch', 'meals': [], 'calcu...","{'id': 3, 'name': 'Lunch', 'meals': [], 'calcu...","{'id': 3, 'name': 'Lunch', 'meals': [], 'calcu...","{'id': 3, 'name': 'Lunch', 'meals': [], 'calcu...","{'id': 3, 'name': 'Lunch', 'meals': [], 'calcu...","{'id': 3, 'name': 'Lunch', 'meals': [], 'calcu...","{'id': 3, 'name': 'Lunch', 'meals': [{'base': ...","{'id': 3, 'name': 'Lunch', 'meals': [{'grams':...","{'id': 3, 'name': 'Lunch', 'meals': [{'mealNam...",...,"{'id': 3, 'name': 'Lunch', 'meals': [{'grams':...","{'id': 3, 'name': 'Lunch', 'meals': [{'mealNam...","{'id': 3, 'name': 'Lunch', 'meals': [{'mealNam...","{'id': 3, 'name': 'Lunch', 'meals': [{'mealNam...","{'id': 3, 'name': 'Lunch', 'meals': [{'mealNam...","{'id': 3, 'name': 'Lunch', 'meals': [{'mealNam...","{'id': 3, 'name': 'Lunch', 'meals': [{'mealNam...","{'id': 3, 'name': 'Lunch', 'meals': [{'grams':...","{'id': 3, 'name': 'Lunch', 'meals': [{'mealNam...","{'id': 3, 'name': 'Lunch', 'meals': [], 'calcu..."
3,"{'id': 4, 'name': 'Snack II', 'meals': [], 'ca...","{'id': 4, 'name': 'Snack II', 'meals': [], 'ca...","{'id': 4, 'name': 'Snack II', 'meals': [], 'ca...","{'id': 4, 'name': 'Snack II', 'meals': [], 'ca...","{'id': 4, 'name': 'Snack II', 'meals': [], 'ca...","{'id': 4, 'name': 'Snack II', 'meals': [], 'ca...","{'id': 4, 'name': 'Snack II', 'meals': [], 'ca...","{'id': 4, 'name': 'Snack II', 'meals': [{'gram...","{'id': 4, 'name': 'Snack II', 'meals': [{'meal...","{'id': 4, 'name': 'Snack II', 'meals': [{'meal...",...,"{'id': 4, 'name': 'Snack II', 'meals': [], 'ca...","{'id': 4, 'name': 'Snack II', 'meals': [{'base...","{'id': 4, 'name': 'Snack II', 'meals': [], 'ca

In [5]:
# Założenie: Kolumny w pliku odpowiadają wymaganym danym
X = df[["glucose", "carbs", "insulin_per_ww"]]
y = df["insulin_dose"]

# Podział na zestawy treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tworzenie modelu XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)

# Trenowanie modelu
model.fit(X_train, y_train)


KeyError: "None of [Index(['glucose', 'carbs', 'insulin_per_ww'], dtype='object')] are in the [columns]"

In [ ]:
# Predykcja na zbiorze testowym
y_pred = model.predict(X_test)

# Ocena modelu
mae = mean_absolute_error(y_test, y_pred)
print(f'"Mean Absolute Error: {mae:.2f}"')

In [ ]:
# **TESTOWANIE MODEL:** Użytkownik może ręcznie wpisać dane
def manual_test():
    glucose = float(input("Podaj aktualny poziom cukru: "))
    carbs = float(input("Podaj ilość spożywanych węglowodanów: "))
    insulin_per_ww = float(input("Podaj dawkę insuliny na 1 WW: "))
    
    input_data = np.array([[glucose, carbs, insulin_per_ww]])
    prediction = model.predict(input_data)[0]
    print(f'Przewidywana dawka insuliny: {prediction:.2f} jednostek')

# Uruchomienie testu manualnego
manual_test()